In [10]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

### Objective: Which quantitative paramater efficiently represents the extent of cyanobacterial contamination in Alberta's lakes and can be used as a surrogate for the level of contamination? (Feature Extraction)

The dataset contains five quantitative features that can indicate the presence of cyanobacterial blooms-

Microcystin LR-equivalent concentration (µg/L)
Total cyanobacterial cell count (cells/mL)
Microcystis mcyE (copies/mL)
Anabaena mcyE (copies/mL)
Microcystis mcyE (copies/mL)
The dataset contains information about three common genera of cyanobacteria that produce the potent toxin, microcystin (MC-LR), implicated in cyanobacterial bloom-linked toxicity (https://doi.org/10.1128/AEM.01058-06). The three genera- Mycocystis, Anabaena (now called Dolichospermum), and Planktothrix are detected in the given data by the expression of their genus-specific Microcystin Synthetase Gene E (mcyE).

In order to extract the most accurate feature to represent contamination in lakes, we compare and contrast all the five bloom indicators in the following section. We do so by examining:

how the mcyE gene expression compares with the other quantitative measures of cyanobacterial blooms- the total cyanobacterial cell counts and the concentration of the microcystin LR (MC-LR) toxin.

the prevalence of the MC producing genera in the lakes from the mcyE gene expression data.

In [2]:
# Import dataset

df = pd.read_csv('dataBloom.csv')
#display(df.info())
display(df.head())
df = df.sort_values('Latitude')
display(df.tail())


,Sample_number,Waterbody_name,Beach_name,Beach_access_num,Latitude,Longitude,Collection_date,Collection_time,MC-LR_conc,Total_cyano_cell_count,...,Turbidity,Color,Water_temp,Wind_direction,Rainfall_24h_bool,Rainfall_24h_mm,EOB,Year,Month,Day
0,M141973,Baptiste Lake,Baptiste Lake Public Beach,8830387.0,54.726988,-113.568914,2014-06-24,15:30:00,NaN,532110.0,...,NaN,Green,20.2,NE,True,NaN,streaks on the surface,2014,6,24
1,M141680,Beaver Lake,Young's Beach,9621938.0,54.753226,-111.925127,2014-07-29,11:30:00,0.52,1793582.0,...,NaN,NaN,NaN,NaN,NaN,NaN,streaks on the surface,2014,7,29
2,M141971,Bonnie Lake,Bonnie Lake Campground Beach,9013424.0,54.146363,-111.881312,2014-07-25,14:00:00,0.07,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,streaks on the surface,2014,7,25
3,M141382,Chestermere Lake,Camp Chestermere Beach,8805982.0,51.020865,-113.818185,2014-06-04,15:00:00,<0.05,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,not apparent,2014,6,4
4,M141386,Chestermere Lake,Camp Chestermere Beach,8805982.0,51.020865,-113.818185,2014-06-11,13:30:00,<0.05,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,not apparent,2014,6,11


,Sample_number,Waterbody_name,Beach_name,Beach_access_num,Latitude,Longitude,Collection_date,Collection_time,MC-LR_conc,Total_cyano_cell_count,...,Turbidity,Color,Water_temp,Wind_direction,Rainfall_24h_bool,Rainfall_24h_mm,EOB,Year,Month,Day
192,M141624,Moonshine Lake,Moonshine Lake Provincial Park,9320045.0,55.889726,-119.229366,2014-07-29,10:30:00,2.88,42017.0,...,NaN,NaN,NaN,NaN,NaN,NaN,streaks on the surface,2014,7,29
669,M151603,Lac Cardinal (Bear Lake),Queen Elizabeth Park Beach,9280133.0,56.219364,-117.699936,2015-07-13,11:15:00,0.24,0.0,...,Moderate,Green,23.3,SE,True,0.1,Particles in water,2015,7,13
1482,M171553,Hutch Lake,Hutch Lake Campground and Day Use,9010033.0,58.765640,-117.369400,2017-08-04,NaN,NaN,2875.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,4
1483,M171554,Hutch Lake,Hutch Lake Campground and Day Use,9010033.0,58.765640,-117.369400,2017-08-04,NaN,NaN,1268.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,4
1484,M171555,Hutch Lake,Hutch Lake Campground and Day Use,9010033.0,58.765640,-117.369400,2017-08-04,NaN,NaN,11133.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,8,4


Extract Quantitative Indicators of harmful blooms

In [3]:
bloomIndicators = df[['Waterbody_name', 'MC-LR_conc', 'Total_cyano_cell_count', 'Microcystis_mcyE', 'Anabaena_mcyE', 'Planktothrix_mcyE', 'Year', 'Month', 'Day']]
print(bloomIndicators.count())
bloomIndicators['MC-LR_conc'] = bloomIndicators['MC-LR_conc'].str.replace(r'(^.*<.*$)', '0').apply(pd.to_numeric)

# Remove entries withh missing data in any of the quantitative features
bloomIndicators = bloomIndicators.dropna(axis=0, how='any')
display(bloomIndicators.count())

# We have 1399 entries for investigating the correlation between the quantitative features
# Convert MC-LR_conc. column dtype to float replacing any values with <0.1 with 0


display(bloomIndicators.info())
display(bloomIndicators)



Waterbody_name            2131
MC-LR_conc                2042
Total_cyano_cell_count    2131
Microcystis_mcyE          1390
Anabaena_mcyE             1390
Planktothrix_mcyE         1390
Year                      2131
Month                     2131
Day                       2131
dtype: int64


C:\Users\radhi\AppData\Local\Temp/ipykernel_22900/3752498384.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  bloomIndicators['MC-LR_conc'] = bloomIndicators['MC-LR_conc'].str.replace(r'(^.*<.*$)', '0').apply(pd.to_numeric)
C:\Users\radhi\AppData\Local\Temp/ipykernel_22900/3752498384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bloomIndicators['MC-LR_conc'] = bloomIndicators['MC-LR_conc'].str.replace(r'(^.*<.*$)', '0').apply(pd.to_numeric)


Waterbody_name            1349
MC-LR_conc                1349
Total_cyano_cell_count    1349
Microcystis_mcyE          1349
Anabaena_mcyE             1349
Planktothrix_mcyE         1349
Year                      1349
Month                     1349
Day                       1349
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1349 entries, 556 to 192
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Waterbody_name          1349 non-null   object 
 1   MC-LR_conc              1349 non-null   float64
 2   Total_cyano_cell_count  1349 non-null   float64
 3   Microcystis_mcyE        1349 non-null   float64
 4   Anabaena_mcyE           1349 non-null   float64
 5   Planktothrix_mcyE       1349 non-null   float64
 6   Year                    1349 non-null   int64  
 7   Month                   1349 non-null   int64  
 8   Day                     1349 non-null   int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 105.4+ KB


None

,Waterbody_name,MC-LR_conc,Total_cyano_cell_count,Microcystis_mcyE,Anabaena_mcyE,Planktothrix_mcyE,Year,Month,Day
556,Forty Mile Reservoir,0.000,22915.0,0.0,0.0,0.0,2015,8,10
59,Forty Mile Reservoir,0.050,8352.0,0.0,0.0,0.0,2014,6,16
60,Forty Mile Reservoir,0.060,0.0,0.0,0.0,0.0,2014,6,23
61,Forty Mile Reservoir,0.070,0.0,0.0,0.0,0.0,2014,7,3
62,Forty Mile Reservoir,0.060,0.0,0.0,0.0,0.0,2014,7,7
...,...,...,...,...,...,...,...,...,...
189,Moonshine Lake,0.080,79094.0,0.0,0.0,0.0,2014,6,17
190,Moonshine Lake,0.160,47110.0,0.0,0.0,0.0,2014,7,2
191,Moonshine Lake,1.620,408709.0,0.0,0.0,0.0,2014,7,15
188,Moonshine Lake,0.063,11765.0,0.0,0.0,0.0,2014,6,2


In [5]:
# Plot a correlation matrix for the various indicators of cyanobacterial blooms
fig = px.scatter_matrix(bloomIndicators,
                        dimensions = ['MC-LR_conc', 'Total_cyano_cell_count', 'Microcystis_mcyE', 'Anabaena_mcyE', 'Planktothrix_mcyE'],
                        color = 'Waterbody_name',
                        height=1000, width= 1200
    )
fig.update_layout(xaxis_type="log",yaxis_type="log")
fig.show()

Since it's possible for waterbodies to have high levels of total cyanobacterial cell counts, but only one of the three species of bacteria present, we should compute these correlations individually for each pair of indicators. 

Calculate the proportion of samples made by the three micocystin producing genera

In [6]:
bloomIndicators = df[['Waterbody_name', 'MC-LR_conc', 'Total_cyano_cell_count', 'Microcystis_mcyE', 'Anabaena_mcyE', 'Planktothrix_mcyE', 'Year', 'Month', 'Day']]
bloomIndicators['MC-LR_conc'] = bloomIndicators['MC-LR_conc'].str.replace(r'(^.*<.*$)', '0').apply(pd.to_numeric)
#print(bloomIndicators.count())

# Samples where only 1 microcystin producing genus is present 
M_Samples = bloomIndicators[bloomIndicators['Microcystis_mcyE'] > 0]
M_SamplesProp = (len(M_Samples)/len(bloomIndicators))*100

A_Samples = bloomIndicators[bloomIndicators['Anabaena_mcyE'] > 0]
A_SamplesProp = (len(A_Samples)/len(bloomIndicators))*100

P_Samples = bloomIndicators[bloomIndicators['Planktothrix_mcyE'] > 0]
P_SamplesProp = (len(P_Samples)/len(bloomIndicators))*100


# Samples where 2 out of 3 microcystin producing genera of are present 
M_ASamples = bloomIndicators[(bloomIndicators['Microcystis_mcyE'] > 0) & (bloomIndicators['Anabaena_mcyE'] > 0)]
M_ASamplesProp = (len(M_ASamples)/len(bloomIndicators))*100

M_PSamples = bloomIndicators[(bloomIndicators['Microcystis_mcyE'] > 0) & (bloomIndicators['Planktothrix_mcyE'] > 0)]
M_PSamplesProp = (len(M_PSamples)/len(bloomIndicators))*100

A_PSamples = bloomIndicators[(bloomIndicators['Anabaena_mcyE'] > 0) & (bloomIndicators['Planktothrix_mcyE'] > 0)]
A_PSamplesProp = (len(A_PSamples)/len(bloomIndicators))*100

# Samples where all three microcystin producing genera of are present 
allGeneraSamples = bloomIndicators[((bloomIndicators['Microcystis_mcyE'] > 0) & (bloomIndicators['Anabaena_mcyE'] > 0)) & 
                                  (bloomIndicators['Planktothrix_mcyE'] > 0)]
A_P_NSamplesProp = (len(allGeneraSamples)/len(bloomIndicators))*100

## Plot the proportions of the samples containing the different microcystin producing genera 
names = ['Microcystis spp.', 'Anabaena spp.', 'Planktothrix', 'Microcystis and Anabaena spp.',
                                 'Microcystis and Planktothrix spp.', 'Planktothrix and Anabaena spp.', 'All Three Genera']
plotData = pd.DataFrame(list(zip(names, [M_SamplesProp, A_SamplesProp, P_SamplesProp, M_ASamplesProp, M_PSamplesProp, A_PSamplesProp, A_P_NSamplesProp])),
                        columns = ['Combination', 'Proportion']
                       )
display(plotData)

fig = px.bar(plotData, x= 'Combination', y='Proportion')
fig.show()


C:\Users\radhi\AppData\Local\Temp/ipykernel_22900/4127828257.py:2: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\radhi\AppData\Local\Temp/ipykernel_22900/4127828257.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Combination,Proportion
0,Microcystis spp.,34.021586
1,Anabaena spp.,5.068043
2,Planktothrix,0.797748
3,Microcystis and Anabaena spp.,3.097137
4,Microcystis and Planktothrix spp.,0.187705
5,Planktothrix and Anabaena spp.,0.046926
6,All Three Genera,0.046926


The Plot indicates that species from the Microcystis genus are more prevalent in the cyanobacterial blooms occuring in Alberta lakes.

Next, plot the correlations for 'Total_cyano_cell_count' with every other quantitative cyanobloom indicator

In [7]:
# Create dataframes for each pair of indicators and
# Remove entries withh missing data in any of the selected features
CellCountVsMCLR = bloomIndicators[['Waterbody_name', 'MC-LR_conc', 'Total_cyano_cell_count']].dropna(axis=0, how='any')
display(CellCountVsMCLR.count())

# Select entries where mcyE levels are over the detection limit(>0)
CellCountVsM_mcyE = bloomIndicators[['Waterbody_name', 'Microcystis_mcyE', 'Total_cyano_cell_count']].dropna(axis=0, how='any')
CellCountVsM_mcyE = CellCountVsM_mcyE[CellCountVsM_mcyE['Microcystis_mcyE'] > 0]
display(CellCountVsM_mcyE.count())

CellCountVsA_mcyE = bloomIndicators[['Waterbody_name', 'Anabaena_mcyE', 'Total_cyano_cell_count']].dropna(axis=0, how='any')
CellCountVsA_mcyE = CellCountVsA_mcyE[CellCountVsA_mcyE['Anabaena_mcyE'] > 0]
display(CellCountVsA_mcyE.count())

CellCountVsP_mcyE = bloomIndicators[['Waterbody_name', 'Planktothrix_mcyE', 'Total_cyano_cell_count']].dropna(axis=0, how='any')
CellCountVsP_mcyE = CellCountVsP_mcyE[CellCountVsP_mcyE['Planktothrix_mcyE'] > 0]
display(CellCountVsP_mcyE.count())



Waterbody_name            2042
MC-LR_conc                2042
Total_cyano_cell_count    2042
dtype: int64

Waterbody_name            725
Microcystis_mcyE          725
Total_cyano_cell_count    725
dtype: int64

Waterbody_name            108
Anabaena_mcyE             108
Total_cyano_cell_count    108
dtype: int64

Waterbody_name            17
Planktothrix_mcyE         17
Total_cyano_cell_count    17
dtype: int64

In [19]:
# Plot 1 Cell Count vs MCLR
corr = stats.pearsonr(CellCountVsMCLR['MC-LR_conc'], CellCountVsMCLR['Total_cyano_cell_count'])
fig = px.scatter(CellCountVsMCLR, x = 'Total_cyano_cell_count', y = 'MC-LR_conc')
# add annotation
fig.add_annotation(dict(font=dict(color='black',size=20),
                                        x=0.2,
                                        y=0.95,
                                        showarrow=False,
                                        text= "Pearson's r: " + str(round(corr[0], 2)) +"\nPVal: " + str(round(corr[1], 2)),
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
fig.show()

## Remove Berry Creek Reservoir and Isle lake from the dataset

df = CellCountVsMCLR[~CellCountVsMCLR['Waterbody_name'].isin(['Berry Creek Reservoir', 'Isle Lake'])]
corr = stats.pearsonr(df['MC-LR_conc'], df['Total_cyano_cell_count'])
fig = px.scatter(df, x = 'Total_cyano_cell_count', y = 'MC-LR_conc',
                log_x=True,log_y=True)
fig.add_annotation(dict(font=dict(color='black',size=20),
                                        x=0.2,
                                        y=0.95,
                                        showarrow=False,
                                        text= "Pearson's r: " + str(round(corr[0], 2)) +"\nPVal: " + str(round(corr[1], 2)),
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
fig.show()


In [16]:
# Plot 2- Cell Count vs Microcystis_mcyE
corr = stats.pearsonr(CellCountVsM_mcyE['Microcystis_mcyE'], CellCountVsM_mcyE['Total_cyano_cell_count'])
fig = px.scatter(CellCountVsM_mcyE, x = 'Total_cyano_cell_count', y = 'Microcystis_mcyE', log_x=True,log_y=True)
fig.add_annotation(dict(font=dict(color='black',size=20),
                                        x=0.2,
                                        y=0.95,
                                        showarrow=False,
                                        text= "Pearson's r: " + str(round(corr[0], 2)) +"\nPVal: " + str(round(corr[1], 2)),
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
fig.show()



In [17]:
# Plot 3- Cell Count vs Anabaena_mcyE
corr = stats.pearsonr(CellCountVsA_mcyE['Anabaena_mcyE'], CellCountVsA_mcyE['Total_cyano_cell_count'])
fig = px.scatter(CellCountVsA_mcyE, x = 'Total_cyano_cell_count', y = 'Anabaena_mcyE',log_x=True,log_y=True)
fig.add_annotation(dict(font=dict(color='black',size=20),
                                        x=0.2,
                                        y=0.95,
                                        showarrow=False,
                                        text= "Pearson's r: " + str(round(corr[0], 2)) +"\nPVal: " + str(round(corr[1], 2)),
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
fig.show()

In [18]:
# Plot 3- Cell Count vs Anabaena_mcyE
corr = stats.pearsonr(CellCountVsP_mcyE['Planktothrix_mcyE'], CellCountVsP_mcyE['Total_cyano_cell_count'])
fig = px.scatter(CellCountVsP_mcyE, x = 'Total_cyano_cell_count', y = 'Planktothrix_mcyE',log_x=True,log_y=True)
               
fig.add_annotation(dict(font=dict(color='black',size=20),
                                        x=0.2,
                                        y=0.95,
                                        showarrow=False,
                                        text= "Pearson's r: " + str(round(corr[0], 2)) +"\nPVal: " + str(round(corr[1], 2)),
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
fig.show()

#### Conclusion: 
The figures suggest that the MC-LR toxin concentration has a strong positive correlation with the microcystis_mcyE expression, but not with the other genera of cyanobacteria. This can potentially be explained by the stark differences in the prevalence the three genera in the samples from the lakes as seen in the figure below.  